# Resource Use

In [28]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os 
import json

In [2]:
# Print the key prefixes to help with any debugging

google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")


Google API Key exists and begins AI


In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
linkedin

'Amir Mohammad Beheshti\n■ Mashhad | ■ amirmohammad.beheshti@gmail.com | ■ 09152622703\nMarital Status: Single | Military Service: Student Exemption | Born: ■■■■/■■/■■\nAbout Me\nExperienced Software Engineer with extensive skills across various programming and software development fields.\nStrong analytical abilities and a talent for finding optimal solutions to complex problems. Passionate about learning\nnew technologies and collaborating in dynamic, creative environments. Proficient in numerous tools and technologies\nrelated to software development and large-scale system architecture.\nAI & Machine Learning\n\x7f Familiar with basic AI concepts and machine learning algorithms\n\x7f Understanding of LLMs and how they work\n\x7f Knowledge of how to implement and fine-tune models\n\x7f Familiar with RAG techniques for information retrieval\n\x7f Experience working with Vector Databases for semantic search\n\x7f Familiar with LangChain and LlamaIndex tools for LLM-based app developmen

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Amir Mohammad Beheshti"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

print(system_prompt)


You are acting as Amir Mohammad Beheshti. You are answering questions on Amir Mohammad Beheshti's website, particularly questions related to Amir Mohammad Beheshti's career, background, skills and experience. Your responsibility is to represent Amir Mohammad Beheshti for interactions on the website as faithfully as possible. You are given a summary of Amir Mohammad Beheshti's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
My name is Amir Mohammad Beheshti. I'm a software engineer with experience in backend development, frontend development, and artificial intelligence. I'm originally from Mashhad, Iran, and I’m passionate about learning new technologies and working in dynamic, creative environments. I enjoy solving complex problems, building efficient systems, and exploring topics like large language

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    model_name = "gemini-2.0-flash"
    print(messages)
    client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    response = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    return response.choices[0].message.content

In [9]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# Tools Use 

In [10]:
# For pushover
import requests


pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")
    
    

def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

Pushover user found and starts with u
Pushover token found and starts with a


In [11]:
push("HEY!!")

Push: HEY!!


In [12]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [13]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}


record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [14]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [15]:
# Tips We Can Call Function With string Of That 
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [16]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [17]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Amir Behehsti"

In [18]:
summary

"My name is Amir Mohammad Beheshti. I'm a software engineer with experience in backend development, frontend development, and artificial intelligence. I'm originally from Mashhad, Iran, and I’m passionate about learning new technologies and working in dynamic, creative environments. I enjoy solving complex problems, building efficient systems, and exploring topics like large language models and machine learning. Outside of work, I like discovering new tools and improving my technical skills through hands-on projects."

In [19]:
linkedin

'Amir Mohammad Beheshti\n■ Mashhad | ■ amirmohammad.beheshti@gmail.com | ■ 09152622703\nMarital Status: Single | Military Service: Student Exemption | Born: ■■■■/■■/■■\nAbout Me\nExperienced Software Engineer with extensive skills across various programming and software development fields.\nStrong analytical abilities and a talent for finding optimal solutions to complex problems. Passionate about learning\nnew technologies and collaborating in dynamic, creative environments. Proficient in numerous tools and technologies\nrelated to software development and large-scale system architecture.\nAI & Machine Learning\n\x7f Familiar with basic AI concepts and machine learning algorithms\n\x7f Understanding of LLMs and how they work\n\x7f Knowledge of how to implement and fine-tune models\n\x7f Familiar with RAG techniques for information retrieval\n\x7f Experience working with Vector Databases for semantic search\n\x7f Familiar with LangChain and LlamaIndex tools for LLM-based app developmen

In [20]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [42]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        print('messages' , messages[len(messages) - 1])
        client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/"  )
        response = client.chat.completions.create(
            model="gemini-2.0-flash",
            messages=messages,
            tools=tools
        )

        finish_reason = response.choices[0].finish_reason
        print("finish_reason" , finish_reason)
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [43]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


messages {'role': 'user', 'content': 'who is ed donner ?'}
finish_reason tool_calls
Tool called: record_unknown_question
Push: Recording Who is Ed Donner? asked that I couldn't answer
messages {'role': 'tool', 'content': '{"recorded": "ok"}', 'tool_call_id': 'function-call-15416429363675392553'}
finish_reason stop
messages {'role': 'user', 'content': 'what is my name ?'}
finish_reason stop
